<a href="https://colab.research.google.com/github/PriyathamVarma/tensorflow/blob/main/Classification_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is for classification using tensorflow created on 30/12/2021

In [ ]:
# this is for specifing the exact version
%tensorflow_version 1.3.0

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.3.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
# imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [31]:
tf.__version__

'1.15.2'

# The aim of this notebook is to demonstrate basic classification using tensorflow 

*   Dataset is Pima Indians diabetes 
*   Tf.estimator API
*   Categorical and continous features
*   Linear Classifier and Dense Neural Network Classifier







In [ ]:
# select the databse from tensorflow basic library
data_frame = pd.read_csv('/content/pima-indians-diabetes.csv')

In [ ]:
columns_list = ['Pregnancies','Glucose',
                'BloodPressure','SkinThickness',
                'Insulin','BMI','DiabetesPedigreeFunction',
                'Age','Class']

In [ ]:
data_frame.columns = columns_list

In [ ]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767 entries, 0 to 766
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               767 non-null    int64  
 1   Glucose                   767 non-null    int64  
 2   BloodPressure             767 non-null    int64  
 3   SkinThickness             767 non-null    int64  
 4   Insulin                   767 non-null    int64  
 5   BMI                       767 non-null    float64
 6   DiabetesPedigreeFunction  767 non-null    float64
 7   Age                       767 non-null    int64  
 8   Class                     767 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


# Normalize the data

In [ ]:
normalized_dataFrame = data_frame[columns_list].apply(lambda x: (x-x.min())/(x.max()-x.min()))

In [ ]:
normalized_dataFrame

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667,0.0
1,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333,1.0
2,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000,0.0
3,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000,1.0
4,0.294118,0.582915,0.606557,0.000000,0.000000,0.381520,0.052519,0.150000,0.0
...,...,...,...,...,...,...,...,...,...
762,0.588235,0.507538,0.622951,0.484848,0.212766,0.490313,0.039710,0.700000,0.0
763,0.117647,0.613065,0.573770,0.272727,0.000000,0.548435,0.111870,0.100000,0.0
764,0.294118,0.608040,0.590164,0.232323,0.132388,0.390462,0.071307,0.150000,0.0
765,0.058824,0.633166,0.491803,0.000000,0.000000,0.448584,0.115713,0.433333,1.0


In [ ]:
# Numeric columns
Pregnancies = tf.feature_column.numeric_column('Pregnancies')
Glucose = tf.feature_column.numeric_column('Glucose')
BloodPressure = tf.feature_column.numeric_column('BloodPressure')
SkinThickness = tf.feature_column.numeric_column('SkinThickness')
Insulin = tf.feature_column.numeric_column('Insulin')
BMI = tf.feature_column.numeric_column('BMI')
DiabetesPedigreeFunction = tf.feature_column.numeric_column('DiabetesPedigreeFunction')
Age = tf.feature_column.numeric_column('Age')

In [ ]:
# Create a bucket
age_bucket = tf.feature_column.bucketized_column(Age,boundaries=[20,30,40,50,60,70,80,90])

In [ ]:
age_bucket

BucketizedColumn(source_column=NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(20, 30, 40, 50, 60, 70, 80, 90))

In [35]:
features_columns = [Pregnancies,Glucose,
                BloodPressure,SkinThickness,
                Insulin,BMI,DiabetesPedigreeFunction,
                Age] 

In [36]:
features_columns

[NumericColumn(key='Pregnancies', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Glucose', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='BloodPressure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SkinThickness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Insulin', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='BMI', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='DiabetesPedigreeFunction', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

# Train and test data

In [37]:
x_data = data_frame.drop('Class',axis=1)

In [38]:
x_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,1,85,66,29,0,26.6,0.351,31
1,8,183,64,0,0,23.3,0.672,32
2,1,89,66,23,94,28.1,0.167,21
3,0,137,40,35,168,43.1,2.288,33
4,5,116,74,0,0,25.6,0.201,30
...,...,...,...,...,...,...,...,...
762,10,101,76,48,180,32.9,0.171,63
763,2,122,70,27,0,36.8,0.340,27
764,5,121,72,23,112,26.2,0.245,30
765,1,126,60,0,0,30.1,0.349,47


In [39]:
labels = data_frame['Class']

In [41]:
labels

0      0
1      1
2      0
3      1
4      0
      ..
762    0
763    0
764    0
765    1
766    0
Name: Class, Length: 767, dtype: int64

In [42]:
# Splitting the data
from sklearn.model_selection import train_test_split
x_train,y_train,x_test,y_test = train_test_split(x_data,labels,test_size = 0.2, random_state = 101)